# Predicting Volatility Using Stock Fundamental Data

Anna Runci & Noah Winton

## Setup & Imports

In [5]:
#!pip install tensorflow -q     
#!pip install -q -U keras-tuner #will always need to install if using Google Colab
#!pip install --upgrade pandas

In [14]:
#general libraries always needed
import pandas as pd
import numpy as np
import warnings
np.warnings = warnings
import math

#plotting required modules
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import FixedLocator, FixedFormatter

#sklearn clustering / unsupervised learning modules
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors

#sklearn modules for preprocessing and transformation
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

#TensorFlow required imports
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

#give yourself the opportunity to visualize the pipeline and models
from sklearn import set_config
set_config(display='diagram')

from pathlib import Path

## Function  Definition

In [8]:
#function provided that plots the learning curve for neural networks
def nn_plot_learning_curve( history ):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    ymin, ymax = [], []
    for x in history.history.keys():
        ymax.append( max(history.history[x]))
        ymin.append( min(history.history[x]))
    plt.gca().set_ylim(min(ymin)*.95, max(ymax)*1.05)
    plt.xlabel("EPOCHS")
    plt.show()

In [10]:
#function to verify the existence of a file in the current working directory and download it if not
import os,urllib, urllib.request, sys, tarfile
def downloadDataResource(file, sourcePath, compressed=None):
    if not os.path.isfile(file):
        try:
            urllib.request.urlretrieve(sourcePath+(compressed if compressed else file),(compressed if compressed else file))
            print("Downloaded", (compressed if compressed else file) )
            if compressed:
                ucomp = tarfile.open(compressed)
                ucomp.extractall()
                ucomp.close()
                print("File uncompressed.")
        except:
            print("ERROR: File", (compressed if compressed else file), "not found. Data source missing.")
    else:
        print("Data resource", file, "already downloaded.")

## Source Data

We will be using compustat data downloaded from our Quant Finance Capstone Project, which contains monthly fundamental financial data of 5 industries within the Communications Services Sector. 

In [18]:
df = pd.read_csv(r"C:\Users\annas\OneDrive - Villanova University\Spring 2024\MIS 3080\data.csv")

C:\Users\annas\AppData\Local\Temp\ipykernel_36672\1964343228.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\annas\OneDrive - Villanova University\Spring 2024\MIS 3080\data.csv")


In [19]:
df

,PDATE,GVKEY,PERMNO,DATE,COMNAM,TICKER,RET,MKTCAP,b_mkt,b_smb,...,ROA,CF,ATG,EY,PE,lev,CAratio,REratio,PDATE_Start,PDATE_End
0,31DEC2005,1164,90284,20060131,M C I INC,MCIP,4.66%,NaN,3.4004,-2.4477,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
1,30NOV2005,1164,90284,20051231,M C I INC,MCIP,-0.65%,"$6,474.50",3.7807,-3.1377,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
2,31OCT2005,1164,90284,20051130,M C I INC,MCIP,-0.20%,"$6,517.16",3.7723,-3.1330,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
3,30SEP2005,1164,90284,20051031,M C I INC,MCIP,0.39%,"$6,530.28",4.0187,-3.3620,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
4,31AUG2005,1164,90284,20050930,M C I INC,MCIP,-0.90%,"$8,332.52",4.4489,-3.6055,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55875,30SEP2022,349530,17324,20221031,NEXTPLAY TECHNOLOGIES INC,NXTP,5.40%,$26.59,1.5293,2.5532,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023
55876,31AUG2022,349530,17324,20220930,NEXTPLAY TECHNOLOGIES INC,NXTP,-20.11%,$24.01,1.5425,2.5463,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023
55877,31JUL2022,349530,17324,20220831,NEXTPLAY TECHNOLOGIES INC,NXTP,-7.43%,$30.05,1.5571,2.4969,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023
55878,30JUN2022,349530,17324,20220731,NEXTPLAY TECHNOLOGIES INC,NXTP,-17.57%,$33.96,1.5340,2.4427,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023


## Prepare Data

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55880 entries, 0 to 55879
Columns: 954 entries, PDATE to PDATE_End
dtypes: float64(920), int64(13), object(21)
memory usage: 406.7+ MB


In [23]:
#dropping blank columns 
df = df.dropna(axis=1, how='all')
df

,PDATE,GVKEY,PERMNO,DATE,COMNAM,TICKER,RET,MKTCAP,b_mkt,b_smb,...,ROA,CF,ATG,EY,PE,lev,CAratio,REratio,PDATE_Start,PDATE_End
0,31DEC2005,1164,90284,20060131,M C I INC,MCIP,4.66%,NaN,3.4004,-2.4477,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
1,30NOV2005,1164,90284,20051231,M C I INC,MCIP,-0.65%,"$6,474.50",3.7807,-3.1377,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
2,31OCT2005,1164,90284,20051130,M C I INC,MCIP,-0.20%,"$6,517.16",3.7723,-3.1330,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
3,30SEP2005,1164,90284,20051031,M C I INC,MCIP,0.39%,"$6,530.28",4.0187,-3.3620,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
4,31AUG2005,1164,90284,20050930,M C I INC,MCIP,-0.90%,"$8,332.52",4.4489,-3.6055,...,0.096023,0.008553,0.376621,-0.003949,0.292810,0.502127,0.533001,-0.235873,31MAR2005,31MAR2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55875,30SEP2022,349530,17324,20221031,NEXTPLAY TECHNOLOGIES INC,NXTP,5.40%,$26.59,1.5293,2.5532,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023
55876,31AUG2022,349530,17324,20220930,NEXTPLAY TECHNOLOGIES INC,NXTP,-20.11%,$24.01,1.5425,2.5463,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023
55877,31JUL2022,349530,17324,20220831,NEXTPLAY TECHNOLOGIES INC,NXTP,-7.43%,$30.05,1.5571,2.4969,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023
55878,30JUN2022,349530,17324,20220731,NEXTPLAY TECHNOLOGIES INC,NXTP,-17.57%,$33.96,1.5340,2.4427,...,-0.150632,-0.220876,NaN,-4.224400,-0.261858,2.010769,0.338436,-0.394895,31MAY2022,31MAY2023


In [25]:
column_names = df.columns.tolist()
column_names 

['PDATE',
 'GVKEY',
 'PERMNO',
 'DATE',
 'COMNAM',
 'TICKER',
 'RET',
 'MKTCAP',
 'b_mkt',
 'b_smb',
 'b_hml',
 'aret',
 'gsector',
 'gind',
 'gsubind',
 'MKTCAP_1',
 'datadate',
 'fyear',
 'tic',
 'cusip',
 'conm',
 'ajex',
 'fyr',
 'acdo',
 'aco',
 'acodo',
 'acominc',
 'acox',
 'acqao',
 'acqcshi',
 'acqgdwl',
 'acqic',
 'acqintan',
 'acqinvt',
 'acqppe',
 'acqsc',
 'act',
 'afudcc',
 'afudci',
 'aldo',
 'am',
 'ano',
 'ao',
 'aocidergl',
 'aociother',
 'aocipen',
 'aocisecgl',
 'aodo',
 'aol2',
 'aoloch',
 'aox',
 'ap',
 'apalch',
 'aqa',
 'aqc',
 'aqd',
 'aqeps',
 'aqi',
 'aqp',
 'aqpl1',
 'aqs',
 'arce',
 'arced',
 'arceeps',
 'at',
 'aul3',
 'bast',
 'bastr',
 'billexce',
 'bkvlps',
 'caps',
 'capsft',
 'capx',
 'capxv',
 'cb',
 'cbi',
 'cdvc',
 'ceiexbill',
 'ceq',
 'ceql',
 'ceqt',
 'ch',
 'che',
 'chech',
 'ci',
 'cibegni',
 'cicurr',
 'cidergl',
 'cimii',
 'ciother',
 'cipen',
 'cisecgl',
 'citotal',
 'cld2',
 'cld3',
 'cld4',
 'cld5',
 'clg',
 'cmp',
 'cogs',
 'cshfd',
 'cs

In [ ]:
#drop columns with more than 5% missing values 